In [1]:
import wandb
# from wandb.keras import WandbCallback
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [5]:
iris = load_iris()
X=iris.data
y=iris.target

In [6]:
y = to_categorical(y,num_classes=3)

# Split dataset
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [8]:
wandb.init(
    project='iris_classification',
    name='iris_tf_experiment',
    config={
        "epochs": 50,
        "batch_size": 16,
        "learning_rate": 0.01,
        "optimizer": "adam"
    }
)
config = wandb.config

wandb: Currently logged in as: arya-22310290 (trials_) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [9]:
model = Sequential([
    Dense(16,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(16,activation='relu'),
    Dense(3,activation='softmax')
])

c:\Users\DELL\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
optimizer = tf.keras.optimizers.Adam(learning_rate=config.learning_rate)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
class WandbMetricsCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        wandb.log({
            "epoch": epoch,
            "train_loss": logs.get("loss"),
            "val_loss": logs.get("val_loss"),
            "train_accuracy": logs.get("accuracy"),
            "val_accuracy": logs.get("val_accuracy")
        })

In [14]:
model.fit(
    X_train,y_train,
    validation_data=(X_val,y_val),
    epochs=config.epochs,
    batch_size=config.batch_size,
    callbacks=[WandbMetricsCallback()]
)

Epoch 1/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.4667 - loss: 0.9471 - val_accuracy: 0.7333 - val_loss: 0.6298
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7000 - loss: 0.6088 - val_accuracy: 0.7667 - val_loss: 0.4319
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8000 - loss: 0.4486 - val_accuracy: 0.9333 - val_loss: 0.3146
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9333 - loss: 0.3276 - val_accuracy: 0.9667 - val_loss: 0.2077
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9667 - loss: 0.2166 - val_accuracy: 0.9667 - val_loss: 0.1280
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9667 - loss: 0.1577 - val_accuracy: 1.0000 - val_loss: 0.0855
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9667 - loss: 0.1170 - val_accuracy: 1.0000 - val_loss: 0.0558
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9667 - loss: 0.0939 - val_accuracy: 1.0000 - val_loss: 0.0467


In [15]:
model.save("iris_model.h5")
wandb.save("iris_model.h5")

['c:\\Users\\DELL\\vs_code\\mlops\\Assign2\\wandb\\run-20251106_002921-zivcuf2m\\files\\iris_model.h5']